In [1]:
from benchmarklib.databases import CliqueDatabase
from benchmarklib import setup_logging

import logging

setup_logging(logging.DEBUG)

In [2]:
#clique_db = CliqueDatabase(db_path="/home/nels/dev/python/tweedledum-research/Python2QCCompiler/benchmarklib/shared/problems/clique.db")
clique_db = CliqueDatabase.shared()

In [3]:
from IPython.display import display
import networkx as nx

problems = clique_db.random_sample(limit=3, n=12)

for p in problems:
    print(p.as_adjacency_matrix(), end="\n\n")


[[0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0.]
 [1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1.]
 [1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]]

[[0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1.]
 [1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0.]]

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0

# Compilers

In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService
from dotenv import load_dotenv
import os

# create XAG Compiler

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
API_INSTANCE = os.getenv("API_INSTANCE", None)
service = QiskitRuntimeService(channel="ibm_cloud", token=API_TOKEN, instance=API_INSTANCE)
backend = service.backend(name="ibm_rensselaer")


In [20]:
from benchmarklib.pipeline import PipelineCompiler, QiskitTranspile
from benchmarklib.pipeline.synthesis import XAGSynthesizer, TruthTableSynthesizer, ClassiqSynthesizer

# compiler is defined by a synthesizer and a series of steps
compiler_xag = PipelineCompiler(
    synthesizer = XAGSynthesizer(),
    # Transpilation is an optional step, but passing a backend & transpile options will do it automatically
    #steps = [QiskitTranspile(backend=backend, optimization_level=3)]
    steps = [], 
    backend = backend, 
    transpile_options = {"optimization_level": 3}
)

compiler_tt = PipelineCompiler(
    synthesizer = TruthTableSynthesizer(),
    steps=[],
    backend = backend, 
    transpile_options = {"optimization_level": 3}
)

# compiler_classiq = PipelineCompiler(
#     synthesizer = ClassiqSynthesizer(),
#     steps = [QiskitTranspile(backend=backend, optimization_level=3)]
# )

In [21]:
from pprint import pprint 

p = problems[1]
result_xag = compiler_xag.compile(problem=p, return_intermediate=True, clique_size=6)
pprint(result_xag)

result_tt = compiler_tt.compile(problem=p, return_intermediate=True, clique_size=6)

# ensure you are authenticated for classiq
#result_classiq = compiler_classiq.compile(problem=p, return_intermediate=True, clique_size=3)

#print("Truth Table Success: ", result_tt.success)
#print("Classiq Success:", result_tt.success)

benchmarklib.pipeline.synthesis.xag - DEBUG - Optimizing XAG...
benchmarklib.pipeline.synthesis.xag - DEBUG - Synthesizing from XAG...
benchmarklib.pipeline.synthesis.xag - DEBUG - Applying optimization passes...


CompilationResult(compiler_name='XAGSynthesizer',
                  success=True,
                  total_time=16.432971000671387,
                  synthesis_metrics=CircuitMetrics(num_qubits=125,
                                                   depth=203,
                                                   gate_count=411,
                                                   entangling_count=407,
                                                   single_qubit_count=4,
                                                   ops=OrderedDict([('cx', 182),
                                                                    ('ccrx_o1',
                                                                     110),
                                                                    ('ccrx_o2',
                                                                     52),
                                                                    ('ccrx',
                                                            

benchmarklib.pipeline.synthesis.truth_table - DEBUG - Simulating to get truth table...
benchmarklib.pipeline.synthesis.truth_table - DEBUG - Synthesizing from truth table...
benchmarklib.pipeline.synthesis.truth_table - DEBUG - Applying optimization passes...


In [22]:
from IPython.display import display
from qiskit.qasm2 import dumps
import pyzx as zx
from qiskit import QuantumCircuit, transpile

print(result_xag.synthesis_circuit.depth())


high_level = result_xag.synthesis_circuit.decompose(reps=2)
low_level = result_xag.final_circuit

circuits = [high_level, low_level]
zx_circuits = list()

for circ in circuits:
    circuit = zx.Circuit.from_qasm(dumps(result_xag.synthesis_circuit.decompose(reps=8)))
    print(circuit.stats)
    # to use simplification rules must be converted to graph
    g = circuit.to_graph()
    zx.full_reduce(g, quiet=False) # simplifies the Graph in-place, and show the rewrite steps taken.

    c_opt = zx.extract_circuit(g.copy())
    qasm = zx.Circuit.to_qasm(c_opt)
    qc_opt = QuantumCircuit.from_qasm_str(qasm)
    zx_circuits.append(qc_opt)


high_level_zx = transpile(zx_circuits[0], backend=backend, optimization_level=3)
low_level_zx = transpile(zx_circuits[1], backend=backend, optimization_level=3)

print(f"High Level Synth Depth: {result_xag.synthesis_circuit.depth()}")
print(f"High Level Synth Depth(ZX -> Tranpilation): {high_level_zx.depth()}")

print()
print(f"Low Level Synth Depth: {low_level.depth}")
print(f"Low Level Synth Depth(High Level-> Transpilation -> ZX -> Transpilation): {low_level_zx.depth()}")

203
<bound method Circuit.stats of Circuit(125 qubits, 0 bits, 5799 gates)>
spider_simp: 2435. 746. 395. 249. 36. 19. 14. 10. 7. 5. 4.  11 iterations
id_simp: 2318.  1 iterations
spider_simp: 2179. 70. 68.  3 iterations
pivot_simp: 399. 33. 11. 4. 3. 1. 1. 1.  8 iterations
lcomp_simp: 6737. 225. 296. 296. 278. 145. 172. 162. 158. 132. 127. 108. 98. 93. 90. 88. 57. 56. 44. 42. 37. 35. 30. 26. 20. 17. 14. 16. 9. 10. 9. 11. 10. 8. 9. 6. 5. 5. 4. 4. 4. 4. 4. 3. 3. 4. 5. 3. 2. 2. 1. 1. 1. 1. 1. 1. 1.  57 iterations
id_simp: 749. 411.  2 iterations
spider_simp: 91. 49. 12. 3. 2.  5 iterations
pivot_simp: 151. 64. 26. 10. 4. 1.  6 iterations
lcomp_simp: 19. 17. 6. 3. 4. 3. 1. 1.  8 iterations
id_simp: 77.  1 iterations
spider_simp: 44. 21. 4. 2. 1.  5 iterations
pivot_simp: 31. 9. 4. 2. 1.  5 iterations
lcomp_simp: 3. 3. 1. 1.  4 iterations
id_simp: 5.  1 iterations
spider_simp: 5.  1 iterations
pivot_simp: 4.  1 iterations
pivot_gadget_simp: 280. 139. 95. 56. 25. 26. 22. 8. 5. 5. 5. 2.  12 i